In [1]:
import numpy as np
import sys, os

In [104]:
class Player:
    
    def __init__(self, first, last, pos, team, pwr, cnt, spd, fld):
        
        # name and position
        self.first = first 
        self.last = last
        self.pos = pos
        self.team = team
        
        # skills
        self.pwr = pwr
        self.cnt = cnt
        self.spd = spd
        self.fld = fld
        
        # stat counts
        self.singles = 0
        self.doubles = 0
        self.triples = 0
        self.walks = 0
        self.atbats = 0
        self.runs = 0
        self.rbis = 0
        self.hmrs = 0
        
        # stat calculations
        self.hits = self.singles + self.doubles + self.triples + self.hmrs
        if self.atbats == 0:
            self.avg = 0.0
        else:
            self.avg = np.round(self.hits/(self.atbats - self.walks), 3)
            
    def print_skills(self):
        return f'pwr={self.pwr} cnt={self.cnt} spd={self.spd} fld={self.fld}'
    
    def hit_list(self):
        return np.array([self.pwr, self.cnt, self.spd])
    
    def print_stats(self):
        
        # stat calculations
        self.hits = self.singles + self.doubles + self.triples + self.hmrs
        if self.atbats == 0:
            self.avg = 0.0
        else:
            self.avg = np.round(self.hits/self.atbats, 3)
            
        return f'hits={self.hits} walks={self.walks} atbat={self.atbats} avg={self.avg} rbi={self.rbis} hmr={self.hmrs}'

    
class Team:
    
    def __init__(self, county, mascot, roster):
        
        # name info
        self.county = county
        self.mascot = mascot
        self.roster = roster

        # stats 
        self.win = 0
        self.loss = 0
        
    def print_record(self):
        return f'{self.win}-{self.loss}'
    
    def print_name(self):
        return f'{self.county} {self.mascot}'

    
class Game:
    
    max_outs = 3
    max_stks = 3
    max_blls = 4
    max_inning = 9
    empty_diamond = [None, None, None]
    
    # dict corresponds to corefficients multiplied by [pwr, cnt, spd] to creat cum probability
    # probabilites correspond to a results after any given pitch
    action_dict = {'S':5*np.array([.6, 1.5, 1]),
                'D':2*np.array([.8, 1.3, 1.2]), 
                'T':1*np.array([1, 1, 1.5]),
                'H':1*np.array([2, .8, .8]),
                'O':12*np.array([1.3, 1.3, .8]),
                'K':8*np.array([1.5, .8, 1]),
                'B':2*np.array([1.5, 1.2, .7])}
    
    def __init__(self, h_team, a_team):
        
        self.rosters = [h_team.roster, a_team.roster]
    
        self.inning = 1
        
        self.atbat = [0, 0]
        self.hits = [0, 0]
        self.runs = [0, 0] 
        self.errs = [0, 0]
    
    # for now treat this as a full at-bat resulting in an action
    def throw_pitch(self, team_num, p, verbose):
        
        p_hit_list = p.hit_list() 
        
        # calculate action probabilities:
        p_dict = {}
        action_norm = 0
        for key in Game.action_dict:
            
            # take inner product of skill list with action vals
            action_weight = np.dot(Game.action_dict[key], p_hit_list)
            action_norm += action_weight
            p_dict[key] = action_weight
        
        # makes weighted action choice
        action = np.random.choice(list(p_dict.keys()), p=[w/action_norm for w in list(p_dict.values())])
        if verbose:  print(action)
        
        return action
    
    # updates diamond and runs base runners advanced by basehits
    # prob = probability of advancing an extra base
    def advance_runners(self, team_num, bases, p_i):
        
        new_diamond = [None, None, None]
        batter = self.rosters[team_num][p_i]
        
        # assign advance probability based on speed of leading runner
        prob = 0
        for x in reversed(self.diamond):
            if x != None:
                
                lead_runner = self.rosters[team_num][x]
                prob = lead_runner.spd/10
                break
        
        # advance based on prob
        advance = np.random.choice([True, False], p=[prob, 1-prob])
        
        # score all runners on bases
        if (bases == 2 and advance) or bases == 3:
            runs = (3 - self.diamond[:].count(None))
            self.runs[team_num] += runs
            batter.rbis += runs
        
        # move runner from 1st to 3rd
        elif (bases == 1 and advance) or (bases == 2 and not advance):
            runs = (2 - self.diamond[1:].count(None))
            self.runs[team_num] += runs
            batter.rbis += runs
            new_diamond[2] = self.diamond[0]

        # advance runners one base
        elif (bases == 0 and advance) or (bases == 1 and not advance):  
            runs = (1 - self.diamond[2:].count(None))
            self.runs[team_num] += runs
            batter.rbis += runs
            new_diamond[2] = self.diamond[1]
            new_diamond[1] = self.diamond[0]
        
        # no advance means only a change if force
        elif bases == 0 and not advance:
            new_diamond = self.diamond
            
        # place batter on appropriate base
        if bases > 0:
            new_diamond[bases-1] = p_i
            
        self.diamond = new_diamond
    
    # takes in action and makes appropriate game updates
    def action_update(self, team_num, action, p_i):
        
        batter = self.rosters[team_num][p_i]
        batter.atbats += 1
    
        if action == 'K':
            self.outs += 1
            
        elif action == 'O':
            self.outs += 1
            
            # runners may advance if less than 3 outs, remove runner from diamond
            if self.outs < Game.max_outs:
                Game.advance_runners(self, team_num, 0, p_i)
                
        elif action == 'B':
            new_diamond = [None, None, None]
            batter.atbats -= 1

            if self.diamond[0] is not None:
                if self.diamond[1] is not None:
                    if self.diamond[2] is not None:
                        
                        self.runs[team_num] += 1
                        batter.rbis += 1
                        
                    new_diamond[2] = self.diamond[1]
                new_diamond[1] = self.diamond[0]
            new_diamond[0] = p_i
            
            self.diamond = new_diamond
            batter.walks += 1
            
        elif action == 'S':
            Game.advance_runners(self, team_num, 1, p_i)
            batter.singles += 1
        
        elif action == 'D':
            Game.advance_runners(self, team_num, 2, p_i)
            batter.doubles += 1
            
        elif action == 'T':
            Game.advance_runners(self, team_num, 3, p_i)
            batter.triples += 1
            
        elif action == 'H':
            runs = 1 + (3 - self.diamond[:].count(None))
            self.runs[team_num] += runs
            batter.rbis += runs
            batter.hmrs += 1
            self.diamond = Game.empty_diamond
            
    def play_game(self, verbose=1):
        
        # iterate through each inning
        while self.inning < Game.max_inning+1 or self.runs[0] == self.runs[1]:
            
            if verbose:
                print('inning = ', self.inning)
            
            # play inning for away team
            for team_num in [1, 0]:
                self.outs = 0
                self.diamond = Game.empty_diamond[:]
                while self.outs < Game.max_outs:

                    if verbose:
                        print(self.diamond)
                    
                    # player index -- who is at bat
                    p_i = self.atbat[team_num]
                    p = self.rosters[team_num][p_i]

                    action = Game.throw_pitch(self, team_num, p, verbose)
                    Game.action_update(self, team_num, action, p_i)

                    # advance order in lineup by one
                    self.atbat[team_num] = (self.atbat[team_num] + 1)%9

                    #print('outs = ', self.outs)
                if team_num == 1 and verbose: print('switch')
    
            if verbose:
                print(self.runs)
                
            self.inning += 1
        
        if self.runs[0] > self.runs[1]:
            return True, self.rosters
        else:
            return False, self.rosters
    


In [105]:
pos_list = ['C', '1B', '2B', '3B', 'SS', 'DH', 'LF', 'RH', 'CF']
county_list = ['Litchfield', 'Hartford', 'Tolland', 'Windham', 
               'Fairfield', 'New Haven', 'Middlesex', 'New London']
mascot_list = ['Leopards', 'Warriors', 'Braves', 'Bandits',
              'Founders', 'Sharks', 'Rebels', 'Whalers']

In [138]:
def round_stat(stat):
    
    if stat < 1:
        stat = 1
    elif stat > 9:
        stat = 9
        
    return stat

def assign_skills():
    
    # accounts for variance in random and weighted assignments
    norm_var = 3 #2.5
    
    # assign power first at uniform dist
    pwr = np.round(np.random.uniform(1, 9), 0)
    
    # assign contact with inverse correlation to pwr 
    cnt_mean = 6.2 - pwr/4
    cnt = np.round(np.random.normal(cnt_mean, norm_var), 0)
    cnt = round_stat(cnt)
        
    # assign speed negative to pwr, positive with contact
    spd_mean = 5.2 - pwr/3 + cnt/4
    spd = np.round(np.random.normal(spd_mean, norm_var), 0)
    spd = round_stat(spd)
        
    # assign fielding as inverse to other stats
    fld_mean = 7 - (pwr + cnt + spd)/7
    fld = np.round(np.random.normal(fld_mean, norm_var), 0)
    fld = round_stat(fld)
    
    skill_list = [pwr, cnt, spd, fld]
    return skill_list

def assign_names(first_list, last_list):
    
    r1 = int(np.random.uniform(0, len(first_list)))
    r2 = int(np.random.uniform(0, len(last_list)))
    
    first = first_list[r1]
    last = last_list [r2]
    
    return first, last


In [139]:
## generate teams and rosters

## test roster
team_list = []

## makes name files into lists
f_first = open("first_names.txt", "r")
first_list= [name.replace('\n','') for name in f_first.readlines()]
f_first.close()

f_last = open("last_names.txt", "r")
last_list= [name.replace('\n','').title() for name in f_last.readlines()]
f_last.close()

for i, county in enumerate(county_list):
    mascot = mascot_list[i]
    team_name = f'{county} {mascot}'
    roster = []
    
    print('\n', county, mascot, 'roster:\n')
    
    for pos in pos_list:

        skill_list = assign_skills()
        first, last = assign_names(first_list, last_list)
        new_player = Player(first, last, pos, team_name, *skill_list)

        roster.append(new_player)
        print(new_player.first, new_player.last, new_player.pos)
        print(new_player.print_skills())
    
    # instantiate new team with full roster
    new_team = Team(county, mascot, roster)
    team_list.append(new_team)


 Litchfield Leopards roster:

Eugene Lewis C
pwr=2.0 cnt=1.0 spd=5.0 fld=8.0
Eric Evans 1B
pwr=4.0 cnt=8.0 spd=2.0 fld=5.0
Scott Ward 2B
pwr=2.0 cnt=8.0 spd=9 fld=7.0
Joseph Anderson 3B
pwr=4.0 cnt=7.0 spd=6.0 fld=1.0
Jordan Bailey SS
pwr=2.0 cnt=4.0 spd=6.0 fld=5.0
Bradley Ward DH
pwr=3.0 cnt=5.0 spd=4.0 fld=9
Roger Thompson LF
pwr=1.0 cnt=8.0 spd=9.0 fld=1
Vincent Coleman RH
pwr=8.0 cnt=1 spd=6.0 fld=7.0
Michael Stewart CF
pwr=6.0 cnt=7.0 spd=8.0 fld=3.0

 Hartford Warriors roster:

Richard Richardson C
pwr=6.0 cnt=7.0 spd=4.0 fld=1.0
Keith Smith 1B
pwr=7.0 cnt=9 spd=3.0 fld=2.0
Kenneth Griffin 2B
pwr=3.0 cnt=1 spd=9.0 fld=4.0
Alexander Turner 3B
pwr=8.0 cnt=3.0 spd=1.0 fld=4.0
Juan Thomas SS
pwr=4.0 cnt=9.0 spd=2.0 fld=4.0
Terry Jones DH
pwr=8.0 cnt=6.0 spd=4.0 fld=1
Jerry Anderson LF
pwr=1.0 cnt=8.0 spd=9 fld=6.0
Dennis Foster RH
pwr=4.0 cnt=4.0 spd=8.0 fld=7.0
Larry Scott CF
pwr=3.0 cnt=8.0 spd=9.0 fld=7.0

 Tolland Braves roster:

Edward Peterson C
pwr=3.0 cnt=5.0 spd=5.0 fld=5.

In [140]:
'''# simulate a test game, at-bat by at-bat

test_game = Game(team_list[0], team_list[1])
result, final_rosters = test_game.play_game()

# return final stats on each player
for player in final_rosters[0]:
    print (player.first, player.last, player.pos)
    print (player.print_skills())
    print (player.print_stats(), '\n')
'''

"# simulate a test game, at-bat by at-bat\n\ntest_game = Game(team_list[0], team_list[1])\nresult, final_rosters = test_game.play_game()\n\n# return final stats on each player\nfor player in final_rosters[0]:\n    print (player.first, player.last, player.pos)\n    print (player.print_skills())\n    print (player.print_stats(), '\n')\n"

In [141]:
# simulate full season of 80 games between teams

team_num = len(team_list)
gamedays = 100

for day in range(gamedays):
    #print('day = ', day)
    for g in range(team_num//2):
        
        # get team indices so that all teams play each other
        t1 = g
        t2 = (team_num//2) + (g+day)%(team_num//2)
        
        # instantiate game between two teams
        game = Game(team_list[t1], team_list[t2])
        
        # update team records based on game outcome
        if game.play_game(verbose=0)[0]:
            team_list[t1].win += 1 
            team_list[t2].loss += 1
        else:
            team_list[t2].win += 1 
            team_list[t1].loss += 1
            
print('finished season! That was quick...')

finished season! That was quick...


In [142]:
# list teams by final season record:
win_idx = np.argsort(np.array([team.win for team in team_list]))

for i in reversed(win_idx):
    print(team_list[i].print_name(), team_list[i].print_record(), '\n')
    
        

Middlesex Rebels 62-38 

Fairfield Founders 57-43 

New Haven Sharks 56-44 

New London Whalers 54-46 

Litchfield Leopards 49-51 

Windham Bandits 44-56 

Tolland Braves 39-61 

Hartford Warriors 39-61 



In [143]:
# look at full team roster and stats for first team

team_idx = 0
print(team_list[team_idx].print_name(), ':\n')

for player in team_list[team_idx].roster:
    
    print (player.first, player.last)
    print (player.print_skills())
    print (player.print_stats(), '\n')

Litchfield Leopards :

Eugene Lewis
pwr=2.0 cnt=1.0 spd=5.0 fld=8.0
hits=149 walks=38 atbat=482 avg=0.309 rbi=75 hmr=19 

Eric Evans
pwr=4.0 cnt=8.0 spd=2.0 fld=5.0
hits=141 walks=25 atbat=479 avg=0.294 rbi=62 hmr=11 

Scott Ward
pwr=2.0 cnt=8.0 spd=9 fld=7.0
hits=143 walks=37 atbat=456 avg=0.314 rbi=59 hmr=12 

Joseph Anderson
pwr=4.0 cnt=7.0 spd=6.0 fld=1.0
hits=138 walks=23 atbat=454 avg=0.304 rbi=79 hmr=10 

Jordan Bailey
pwr=2.0 cnt=4.0 spd=6.0 fld=5.0
hits=157 walks=33 atbat=435 avg=0.361 rbi=88 hmr=16 

Bradley Ward
pwr=3.0 cnt=5.0 spd=4.0 fld=9
hits=128 walks=26 atbat=436 avg=0.294 rbi=73 hmr=9 

Roger Thompson
pwr=1.0 cnt=8.0 spd=9.0 fld=1
hits=155 walks=35 atbat=416 avg=0.373 rbi=97 hmr=18 

Vincent Coleman
pwr=8.0 cnt=1 spd=6.0 fld=7.0
hits=109 walks=33 atbat=407 avg=0.268 rbi=80 hmr=20 

Michael Stewart
pwr=6.0 cnt=7.0 spd=8.0 fld=3.0
hits=123 walks=28 atbat=405 avg=0.304 rbi=80 hmr=15 



In [144]:
# rank all players by stat of interest

def rank_by_stat(stat, teams = team_list):
    
    all_players = []
    all_stats = []
    for team in team_list:
        for player in team.roster:
            
            all_players.append(player)
            all_stats.append(getattr(player, stat))
            
    sort_idx = np.argsort(all_stats)
    players_sorted = [all_players[i] for i in reversed(sort_idx)]
    
    # return sorted player objects
    return players_sorted

# see attributes in player class for stats to rank by

stat = 'hmrs'
ranked_players = rank_by_stat(stat)

for player in ranked_players:
    print ( f'{player.first} {player.last}\n{player.team}'
            f'\n{player.print_skills()}\n{player.print_stats()}\n')
    

William Adams
Tolland Braves
pwr=6.0 cnt=1.0 spd=4.0 fld=4.0
hits=140 walks=38 atbat=465 avg=0.301 rbi=89 hmr=32

Bobby Allen
New Haven Sharks
pwr=9.0 cnt=9 spd=4.0 fld=1
hits=132 walks=33 atbat=483 avg=0.273 rbi=95 hmr=26

Gary Perez
Middlesex Rebels
pwr=5.0 cnt=2.0 spd=6.0 fld=9
hits=130 walks=26 atbat=459 avg=0.283 rbi=88 hmr=26

Joshua Gray
New London Whalers
pwr=5.0 cnt=2.0 spd=1 fld=4.0
hits=135 walks=43 atbat=461 avg=0.293 rbi=98 hmr=26

Andrew Gray
Fairfield Founders
pwr=8.0 cnt=4.0 spd=4.0 fld=4.0
hits=138 walks=36 atbat=458 avg=0.301 rbi=99 hmr=24

Jack Nelson
Tolland Braves
pwr=6.0 cnt=3.0 spd=1 fld=3.0
hits=111 walks=31 atbat=413 avg=0.269 rbi=84 hmr=23

Lawrence Perry
Middlesex Rebels
pwr=7.0 cnt=5.0 spd=2.0 fld=1
hits=120 walks=29 atbat=421 avg=0.285 rbi=75 hmr=23

Zachary Alexander
Fairfield Founders
pwr=7.0 cnt=7.0 spd=4.0 fld=5.0
hits=136 walks=35 atbat=448 avg=0.304 rbi=89 hmr=22

Terry Hughes
New Haven Sharks
pwr=7.0 cnt=1.0 spd=9 fld=6.0
hits=120 walks=30 atbat=442 